Setup and load data

In [1]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
import pandas as pd

import shap
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import get_paths, set_seed
from src.io import read_table

set_seed(42)
paths = get_paths()

# Load processed datasets
X = read_table(paths.processed / "X_radiomics.csv")
y_days = read_table(paths.processed / "y_days.csv")["days_post_btx"].astype(float)

print("Loaded datasets:")
print(f"- X_radiomics: {X.shape}")
print(f"- y_days:      {y_days.shape} | min={y_days.min()} max={y_days.max()}")

# Output folder (NOT versioned in GitHub; included in Zenodo release)
SHAP_DIR = paths.root / "supplementary" / "shap"
SHAP_DIR.mkdir(parents=True, exist_ok=True)
print(f"SHAP outputs will be saved to: {SHAP_DIR.resolve()}")

Loaded datasets:
- X_radiomics: (571, 105)
- y_days:      (571,) | min=0.0 max=14.0
SHAP outputs will be saved to: C:\Users\modre\Documents\masseter\supplementary\shap


Fit LightGBM regressor

In [2]:
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

reg = Pipeline([
    ("scaler", MinMaxScaler()),
    ("reg", lgb.LGBMRegressor(
        random_state=42,
        n_estimators=1000,
    ))
])

reg.fit(X, y_days)

# Convenience handles
scaler = reg.named_steps["scaler"]
lgbm_model = reg.named_steps["reg"]
X_scaled = pd.DataFrame(scaler.transform(X), columns=X.columns)

print("Model trained for SHAP regression:")
print(f"- n_samples: {len(X_scaled)}")
print(f"- n_features: {X_scaled.shape[1]}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001022 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19285
[LightGBM] [Info] Number of data points in the train set: 571, number of used features: 104
[LightGBM] [Info] Start training from score 5.490368
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

Compute SHAP values

In [3]:
import numpy as np

# Background for TreeExplainer (controls runtime)
BG_N = 200
EXPLAIN_N = len(X_scaled)  # set e.g. 400 for faster runs

rng = np.random.default_rng(42)
bg_idx = rng.choice(len(X_scaled), size=min(BG_N, len(X_scaled)), replace=False)
X_bg = X_scaled.iloc[bg_idx]

X_explain = X_scaled.iloc[:EXPLAIN_N]

explainer = shap.TreeExplainer(lgbm_model, data=X_bg, feature_perturbation="interventional")
shap_values = explainer.shap_values(X_explain)  # regression -> (n_samples, n_features)

print("SHAP regression values shape:", getattr(shap_values, "shape", None))

 97%|=================== | 552/571 [00:16<00:00]       

SHAP regression values shape: (571, 105)


Global feature importance CSV (mean |SHAP|)

In [4]:
import numpy as np
import pandas as pd

if not (isinstance(shap_values, np.ndarray) and shap_values.ndim == 2):
    raise RuntimeError(f"Unexpected SHAP output for regression: {type(shap_values)} / {getattr(shap_values, 'shape', None)}")

abs_mean = np.mean(np.abs(shap_values), axis=0)  # (n_features,)

imp_global = (
    pd.DataFrame({"feature": X_explain.columns, "mean_abs_shap": abs_mean})
    .sort_values("mean_abs_shap", ascending=False)
    .reset_index(drop=True)
)

out_csv = SHAP_DIR / "shap_regression_feature_importance_global.csv"
imp_global.to_csv(out_csv, index=False)

print(f"Saved: {out_csv}")
display(imp_global.head(20))

Saved: c:\Users\modre\Documents\masseter\supplementary\shap\shap_regression_feature_importance_global.csv


,feature,mean_abs_shap
0,shapeMaximum2DDiameterRow_delta,1.789702
1,shapeSphericity_delta,0.679483
2,shapeSurfaceVolumeRatio_delta,0.614155
3,ngtdmCoarseness_delta,0.331428
4,firstorderUniformity_delta,0.281091
5,glrlmLongRunHighGrayLevelEmphasis_delta,0.249289
6,glszmSizeZoneNonUniformity_delta,0.226301
7,gldmDependenceVariance_delta,0.225036
8,shapeMeshVolume_delta,0.191144
9,glszmLargeAreaLowGrayLevelEmphasis_delta,0.161657


Summary bar plot

In [5]:
plt.figure()
shap.summary_plot(
    shap_values,
    X_explain,
    plot_type="bar",
    show=False,
    max_display=30
)

out_png = SHAP_DIR / "shap_regression_summary_bar.png"
plt.tight_layout()
plt.savefig(out_png, dpi=300, bbox_inches="tight")
plt.close()

print(f"Saved: {out_png}")

C:\Users\modre\AppData\Local\Temp\ipykernel_54952\921808952.py:2: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(


Saved: c:\Users\modre\Documents\masseter\supplementary\shap\shap_regression_summary_bar.png


Beeswarm plot (global)

In [6]:
plt.figure()
shap.summary_plot(
    shap_values,
    X_explain,
    show=False,
    max_display=30
)

out_png = SHAP_DIR / "shap_regression_beeswarm.png"
plt.tight_layout()
plt.savefig(out_png, dpi=300, bbox_inches="tight")
plt.close()

print(f"Saved: {out_png}")


C:\Users\modre\AppData\Local\Temp\ipykernel_54952\1273229547.py:2: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(


Saved: c:\Users\modre\Documents\masseter\supplementary\shap\shap_regression_beeswarm.png
